In [38]:
import os
import shutil
from IPython import get_ipython

from PIL import Image, ImageOps
import imageio

In [39]:
from diagrams import Diagram, Cluster
from diagrams.aws.compute import ECS, EC2
from diagrams.onprem.container import Docker

In [40]:
notebook_path = get_ipython().starting_dir
output_folder = os.path.join(notebook_path, "diagrams")

shutil.rmtree(output_folder, ignore_errors=True)
os.makedirs(output_folder)

In [41]:
big_name = "ECS Deployment"

In [42]:
name = big_name + " - 1. Initial State"
with Diagram(name, filename=os.path.join(output_folder, name), show=False, direction="TB"):
    with Cluster("ECS Cluster"):
        ecs_service = ECS("ECS Service")
        containers = [Docker("Container 1"), Docker("Container 2")]

    ec2 = EC2("EC2 Capacity")

    ecs_service >> containers
    ec2 >> ecs_service

# This will save the diagram as 'ecs_deployment_initial_state.png'

In [43]:
name = big_name + " - 2. Deploy New Container"
with Diagram(name, os.path.join(output_folder, name), show=False, direction="TB"):
    with Cluster("ECS Cluster"):
        ecs_service = ECS("ECS Service")
        # Two original containers and one new version container
        containers = [Docker("Container 1 - Old"), Docker("Container 2 - Old"), Docker("Container 3 - New")]

    ec2 = EC2("EC2 Capacity")

    ecs_service >> containers
    ec2 >> ecs_service

# This will save the diagram as 'ecs_deployment_deploy_new_container.png'

In [44]:
name = big_name + " - 3. Decommission Old Container"
with Diagram(name, os.path.join(output_folder, name), show=False, direction="TB"):
    with Cluster("ECS Cluster"):
        ecs_service = ECS("ECS Service")
        # One old container and one new version container, after decommissioning one old container
        containers = [Docker("Container 1 - Old"), Docker("Container 2 - New")]

    ec2 = EC2("EC2 Capacity")

    ecs_service >> containers
    ec2 >> ecs_service

# This will save the diagram as 'ecs_deployment_decommission_old_container.png'

In [45]:
name = big_name + " - 4. Rollout Continuation"
with Diagram(name, os.path.join(output_folder, name), show=False, direction="TB"):
    with Cluster("ECS Cluster"):
        ecs_service = ECS("ECS Service")
        # Two new version containers after decommissioning the last old container
        containers = [Docker("Container 1 - New"), Docker("Container 2 - New")]

    ec2 = EC2("EC2 Capacity")

    ecs_service >> containers
    ec2 >> ecs_service

# This will save the diagram as 'ecs_deployment_rollout_continuation.png'

In [58]:
def find_largest_dimensions(image_folder, file_names):
    """ Find the largest width and height among all images """
    max_width, max_height = 0, 0
    for file_name in file_names:
        with Image.open(os.path.join(image_folder, file_name)) as img:
            if img.width > max_width:
                max_width = img.width
            if img.height > max_height:
                max_height = img.height
    return max_width, max_height


def pad_image_to_size(img, size):
    """ Pad image with white background to match the given size """
    padded_img = ImageOps.pad(img, size, color="white")
    return padded_img


def create_fading_gif(image_folder, output_gif, duration=5, fade_duration=2, static_duration=200, loop=0):
    images = []
    filenames = sorted([img for img in os.listdir(image_folder) if img.endswith(".png")])

    # Find the largest image size
    max_width, max_height = find_largest_dimensions(image_folder, filenames)
    largest_size = (max_width, max_height)

    for i in range(len(filenames) - 1):
        img1 = Image.open(os.path.join(image_folder, filenames[i]))
        img2 = Image.open(os.path.join(image_folder, filenames[i + 1]))

        # Pad images to the same size
        img1 = pad_image_to_size(img1, largest_size)
        img2 = pad_image_to_size(img2, largest_size)

        # Display static image for a duration
        for _ in range(static_duration):
            images.append(img1)

        # Create fading effect
        for t in range(fade_duration):
            alpha = t / fade_duration
            blended = Image.blend(img1, img2, alpha)
            images.append(blended)

    # Add and display the last image statically
    last_img = Image.open(os.path.join(image_folder, filenames[-1]))
    last_img = pad_image_to_size(last_img, largest_size)
    for _ in range(static_duration):
        images.append(last_img)

    # Save to GIF with infinite loop
    imageio.mimsave(output_gif, images, duration=duration, loop=loop)


image_folder = output_folder
output_gif = f'{big_name}.gif'  # Output GIF file name
create_fading_gif(image_folder, os.path.join(image_folder, output_gif))